<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/Pivot_Tables_for_OLAP_Cube_Python_Superstore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://4.bp.blogspot.com/-gbL5nZDkpFQ/XScFYwoTEII/AAAAAAAAAGY/CcVb_HDLwvs2Brv5T4vSsUcz7O4r2Q79ACK4BGAYYCw/s1600/kk3-header00-beta.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#SuperStore
Super Store Order Data https://drive.google.com/file/d/13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ/view?usp=sharing <br>
Original Data from [Tableau](https://community.tableau.com/docs/DOC-1236)

In [1]:
#!wget -O SS_Orders2.csv 'https://drive.google.com/uc?export=download&id=13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ'
#!gdown https://drive.google.com/uc?id=13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ
!wget -q -O SS_Orders.csv 'https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/SS_Orders.csv'

In [2]:
import numpy as np
import pandas as pd

In [3]:
dfSS = pd.read_csv('SS_Orders.csv', delimiter = ',')
dfSS.head(10)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,12/06/2016,16/06/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,11/10/2015,18/10/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,11/10/2015,18/10/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
5,6,CA-2014-115812,09/06/2014,14/06/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,FUR-FU-10001487,Furniture,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,48.8600,7,0.00,14.1694
6,7,CA-2014-115812,09/06/2014,14/06/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,OFF-AR-10002833,Office Supplies,Art,Newell 322,7.2800,4,0.00,1.9656
7,8,CA-2014-115812,09/06/2014,14/06/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,TEC-PH-10002275,Technology,Phones,Mitel 5320 IP Phone VoIP phone,907.1520,6,0.20,90.7152
8,9,CA-2014-115812,09/06/2014,14/06/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,OFF-BI-10003910,Office Supplies,Binders,DXL Angle-View Binders with Locking Rings by S...,18.5040,3,0.20,5.7825
9,10,CA-2014-115812,09/06/2014,14/06/2014,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,OFF-AP-10002892,Office Supplies,Appliances,Belkin F5C206VTEL 6 Outlet Surge,114.9000,5,0.00,34.4700


In [4]:
dfSS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

Pivot <br>
https://dfrieds.com/data-analysis/pivot-table-python-pandas.html

In [ ]:
#Create the pivot table
dfSSPivot = pd.pivot_table(data=dfSS,index='Segment',columns='Category',values='Sales', aggfunc='sum').copy()
dfSSPivot

Category,Furniture,Office Supplies,Technology
Segment,,,
Consumer,391049.3120,363952.136,406399.897
Corporate,229019.7858,230676.462,246450.119
Home Office,121930.6975,124418.434,183304.017


In [ ]:
#Calculate the Row Totals
dfSSPivot['RowTotals'] = (dfSSPivot['Furniture']+dfSSPivot['Office Supplies']+dfSSPivot['Technology']).round(2)
dfSSPivot

Category,Furniture,Office Supplies,Technology,RowTotals
Segment,,,,
Consumer,391049.3120,363952.136,406399.897,1161401.35
Corporate,229019.7858,230676.462,246450.119,706146.37
Home Office,121930.6975,124418.434,183304.017,429653.15


In [ ]:
#Create the column totals, transpose the columns and rows
dfColTotals = dfSSPivot.sum().round(2).to_frame().T
#dfColTotals.rename(index={0:'ColTotals'},inplace=True)
dfColTotals

Category,Furniture,Office Supplies,Technology,RowTotals
0,741999.8,719047.03,836154.03,2297200.87


In [ ]:
dfSSPivot2 = dfSSPivot.append(dfColTotals).copy()
dfSSPivot2

Category,Furniture,Office Supplies,Technology,RowTotals
Consumer,391049.3120,363952.136,406399.897,1161401.35
Corporate,229019.7858,230676.462,246450.119,706146.37
Home Office,121930.6975,124418.434,183304.017,429653.15
0,741999.8000,719047.030,836154.030,2297200.87


In [ ]:
dfSSPivot2.rename(index={0:'ColTotals'},inplace=True)
dfSSPivot2

Category,Furniture,Office Supplies,Technology,RowTotals
Consumer,391049.3120,363952.136,406399.897,1161401.35
Corporate,229019.7858,230676.462,246450.119,706146.37
Home Office,121930.6975,124418.434,183304.017,429653.15
ColTotals,741999.8000,719047.030,836154.030,2297200.87


##Pivot Tables in SQL
A cross tab with the same data is available in [this Colab Notebook](https://github.com/Praxis-QR/RDWH/blob/2d97c03428e59b1e5b9a77a4630887dc76c50d00/Cross_Tab_with_OLAP_Cube_SQL_Superstore.ipynb) 

#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)

#eCommerce Data <br>
This data has too few dimensions to be made into a pivot table <br>
Original Data https://www.kaggle.com/carrie1/ecommerce-data <br>
Same data in Google Drive https://drive.google.com/open?id=1K_rQftggePnII3K6613MumhKo3dTMO5r <br>
10% data in Google Drive https://drive.google.com/file/d/1YjCMPc4vUuXvWik1u4p5aP-THJUJjvDX/view?usp=sharing

In [ ]:
!gdown https://drive.google.com/uc?id=1YjCMPc4vUuXvWik1u4p5aP-THJUJjvDX

Downloading...
From: https://drive.google.com/uc?id=1YjCMPc4vUuXvWik1u4p5aP-THJUJjvDX
To: /content/eCommerce02.csv
100% 917k/917k [00:00<00:00, 61.9MB/s]


In [ ]:
dfeCom = pd.read_csv('eCommerce02.csv', delimiter = ',')
dfeCom.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,C544414,22960,JAM MAKING SET WITH JARS,-2,2/18/2011 14:54,3.75,13408.0,United Kingdom
1,555276,48111,DOORMAT 3 SMILEY CATS,1,6/1/2011 17:28,15.79,NaN,United Kingdom
2,575656,22952,60 CAKE CASES VINTAGE CHRISTMAS,48,11/10/2011 14:29,0.55,13319.0,United Kingdom
3,571636,20674,GREEN POLKADOT BOWL,16,10/18/2011 11:41,1.25,13509.0,United Kingdom
4,576657,22556,PLASTERS IN TIN CIRCUS PARADE,12,11/16/2011 11:03,1.65,12720.0,Germany
5,569823,23298,SPOTTY BUNTING,1,10/6/2011 12:15,4.95,16895.0,United Kingdom
6,570185,21090,WET/MOULDY,-192,10/7/2011 14:56,0.00,NaN,United Kingdom
7,574943,17091J,VANILLA INCENSE IN TIN,36,11/8/2011 7:52,0.38,13026.0,United Kingdom
8,561902,22909,SET OF 20 VINTAGE CHRISTMAS NAPKINS,96,7/31/2011 15:50,0.85,17404.0,Sweden
9,550499,22138,BAKING SET 9 PIECE RETROSPOT,1,4/18/2011 15:16,10.79,NaN,United Kingdom


In [ ]:
dfeCom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10838 entries, 0 to 10837
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    10838 non-null  object 
 1   StockCode    10838 non-null  object 
 2   Description  10809 non-null  object 
 3   Quantity     10838 non-null  int64  
 4   InvoiceDate  10838 non-null  object 
 5   UnitPrice    10838 non-null  float64
 6   CustomerID   8057 non-null   float64
 7   Country      10838 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 677.5+ KB


In [ ]:
#Create the pivot table
dfeComPivot = pd.pivot_table(data=dfeCom,index='StockCode',columns='Country',values='Quantity', aggfunc='sum').copy()
dfeComPivot

Country,Australia,Austria,Bahrain,Belgium,Brazil,Canada,Channel Islands,Cyprus,Czech Republic,Denmark,EIRE,Finland,France,Germany,Greece,Hong Kong,Iceland,Israel,Italy,Japan,Lithuania,Malta,Netherlands,Norway,Poland,Portugal,Saudi Arabia,Singapore,Spain,Sweden,Switzerland,USA,United Arab Emirates,United Kingdom,Unspecified
StockCode,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
10125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,170.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
10133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
10135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0,NaN
11001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-176.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DCGS0069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.0,NaN
DOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN
M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,-8.0,NaN
